In [ ]:
from google.cloud import bigquery
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots



In [17]:

time = '2025-06-30'
SQL_QUERY =  f"""
SELECT
  sso_id,additional_info,campaign_details,campaign_name,platform,event_time,record_type
FROM
  `htz-common.raw_data.requests` 
WHERE
  TIMESTAMP_TRUNC(event_time, DAY) >= TIMESTAMP('{time}')
  AND page_type = 'Chatbot'
  AND campaign_name in ('response answer','open question')
  -- AND sso_id IS NOT NULL
ORDER BY
  event_time DESC;
"""
client = bigquery.Client()
query_job = client.query(SQL_QUERY)

results = query_job.result() 
df_all = results.to_dataframe()
# add index
df_all['index'] = df_all.index
df_all['event_time_temp'] = df_all['event_time'].dt.strftime('%Y-%m-%d %H:%M')
df_all['event_time'] = df_all['event_time'].dt.strftime('%Y-%m-%d %H:%M:%S')
df = df_all.drop_duplicates(subset=['sso_id', 'campaign_details','event_time'], keep='last')
response = df[df['campaign_name'] == 'response answer'].drop_duplicates(subset=['sso_id','campaign_details'], keep='last')
df_f = pd.concat([df[df['campaign_name'] == 'open question'], response], ignore_index=True)
df_f = df_f.drop(columns=['event_time_temp'])
print(df_f.shape[0])
df_f = df_f.reset_index(drop=True)
df = df_f.copy()
df.head(2)

/Users/roeishlezinger/Desktop/movie-recommendation-chatbot/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


22


,sso_id,additional_info,campaign_details,campaign_name,platform,event_time,record_type,index
0,8010962861,"[{'key': 'session_id', 'value': '151'}]",היוש,open question,Desktop,2025-06-30 08:31:35,action,1
1,9037014398,"[{'key': 'session_id', 'value': '067b528f-b885...",אפשר דוקו טוב?,open question,App,2025-06-30 08:31:01,action,4


In [18]:
# preprocess the data

df['parsed_data'] = df['additional_info'].apply(lambda x: {item['key']: item['value'] for item in x})
# add index to parsed data
expanded_df = pd.json_normalize(df['parsed_data'].tolist())

expanded_df = expanded_df.drop(columns=['sso_id'])
df_cleaned = df.drop(columns=['additional_info', 'parsed_data'])
df = pd.concat([df_cleaned, expanded_df], axis=1)
df['id'] = df['sso_id'].astype(str) + '_' + df['session_id'].astype(str)

df.head(2)

,sso_id,campaign_details,campaign_name,platform,event_time,record_type,index,session_id,total_time,rag_speed,...,article_ids_1,error,remaining_user_messages,troll_triggered,conversation_key,genres,regenerate,media_type,streaming_platforms,id
0,8010962861,היוש,open question,Desktop,2025-06-30 08:31:35,action,1,151,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8010962861_151
1,9037014398,אפשר דוקו טוב?,open question,App,2025-06-30 08:31:01,action,4,067b528f-b885-4d36-be53-a9dccb6f6483,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9037014398_067b528f-b885-4d36-be53-a9dccb6f6483


In [19]:
df['id'].value_counts().sort_values(ascending=False).head(10)

id
50000227213_16                                     4
9037857899_7                                       4
8010962861_148                                     4
8010962861_151                                     2
9037014398_067b528f-b885-4d36-be53-a9dccb6f6483    2
50000227213_15                                     2
9037857899_4                                       2
8010962861_150                                     1
3057469657_51                                      1
Name: count, dtype: int64

In [31]:
df_all[df_all['sso_id'] == 3057469657]

,sso_id,additional_info,campaign_details,campaign_name,platform,event_time,record_type,index,event_time_temp
15,3057469657,"[{'key': 'total_time', 'value': '4.94688344001...",<strong>גילי איזיקוביץ</strong> ממליצה על הסרט...,response answer,Mobile,2025-06-30 07:23:46,impression,15,2025-06-30 07:23
16,3057469657,"[{'key': 'total_time', 'value': '4.94688344001...",<strong>גילי איזיקוביץ</strong> ממליצה על הסרט...,response answer,Mobile,2025-06-30 07:23:40,impression,16,2025-06-30 07:23


In [20]:
def parse_conversation(ids:str,time:bool) -> str:
    """
    Parse the conversation from the DataFrame based on the provided id.
    """
    conversation = df[df['id'] == ids]
    conversation = conversation.sort_values(by='event_time', ascending=True)
    if conversation.empty:
        return "No conversation found for this ID."
    
    conversation_text = []
    for _, row in conversation.iterrows():
        if row['campaign_name'] == 'open question':
            if time:
                text = f"{row['event_time']} - USER: {row['campaign_details']}"
            else:
                text = f"USER: {row['campaign_details']}"
        elif row['campaign_name'] == 'response answer':
            if time:
                text = f"{row['event_time']} - LLM: {row['campaign_details']}"
            else:
                text = f"LLM: {row['campaign_details']}"
        conversation_text.append(text)
    
    return "\n".join(conversation_text)

txt = parse_conversation('9037857899_7', True)
print(txt)

2025-06-30 06:00:24 - USER: ארצה המלצה על סרט על מלחמת העולם השניה
2025-06-30 06:00:28 - LLM: <strong>אורון שמיר</strong> ממליץ על הסרט <strong>השועל</strong>, סרט גרמני שעלילתו מתרחשת במלחמת העולם השנייה, והוא זמין לצפייה ב-Yes VOD. הסרט עוסק בפרנץ, חייל אוסטרי שגויס לצבא הגרמני ומוצא גור שועלים פצוע בצרפת הכבושה, בו הוא מחליט לטפל. <strong>גילי איזיקוביץ</strong> ממליצה על הסרט התיעודי <strong>חושניה – סודות מלחמה</strong> ששודר ב-HOT 8, ומתאר את טבח 17 חיילי צוותים טכניים של חטיבה 188 שנלקחו בשבי ונרצחו במלחמת יום הכיפורים.האם תרצו המלצות נוספות על סרטים ממלחמת העולם השנייה, אולי בז'אנר אחר, או שאתם מחפשים סרט דוקומנטרי על נושא היסטורי אחר?
2025-06-30 06:01:31 - USER: רק מלחמת העולם השניה לא מלחמות אחרות
2025-06-30 06:01:36 - LLM: בטח. <strong>אורון שמיר</strong> ממליץ על הסרט <strong>במערב אין כל חדש</strong>, עיבוד חדש לקלאסיקת המלחמה על חוויותיו של חייל גרמני בחזית המערבית במלחמת העולם הראשונה (לא השנייה). הסרט זמין בנטפליקס. <strong>נירית אנדרמן</strong> ממליצה על הסרט התיעודי <

In [21]:
from pydantic import BaseModel, Field

class ReviewChat(BaseModel):
    # ---- quality -----
    quality: int = Field(..., description="Quality rating of the chat conversation, from 1 to 5, where 1 is poor and 5 is excellent.")
    feedback: str = Field(..., description="Feedback on the chat conversation.")
    politeness: int = Field(..., description="Politeness rating of the LLM response, from 1 to 5, where 1 is rude and 5 is polite.")
    knowledge_from_RAG: bool = Field(..., description="Whether the LLM response used knowledge from RAG (True or False).")
    answer_relevance: int = Field(..., description="Relevance of the LLM response to the user's question, from 1 to 5, where 1 is not relevant and 5 is highly relevant.")
    # ---- LLM beahavior -----
    not_reveal_system_info: bool = Field(..., description="Whether the LLM response did not reveal any system information (True or False).")
    # prompt_enjected: bool = Field(..., description="Whether the LLM response was prompted to inject system information (True or False).")
    # ---- LLM breakdown -----
    hallucination: bool = Field(..., description="Whether the LLM response contained hallucinations (True or False). for ")

In [22]:
from google import genai
import os

model_name = "gemini-2.5-pro"
system_instruction = """
You are an LLM as a judge designed to test the quality of the output of a RAG-based chatbot for movie and series recommendations. 
You will receive parameters for examining the conversation between the user and the language model, 
and you must return an answer as to whether the correspondence meets the criteria you received.
"""

client = genai.Client(api_key=os.environ.get("GOOGLE_API_KEY"))
temp = client.models.generate_content(
    model=model_name,
    contents=txt,
                config={
            'response_mime_type': 'application/json',
            'response_schema': ReviewChat,
            'system_instruction': system_instruction,
        },
)



In [23]:
temp.parsed

ReviewChat(quality=1, feedback='הצ\'אטבוט נכשל בהבנת הבקשה של המשתמש. בתשובה הראשונה, הוא המליץ על סרט על מלחמת יום כיפור כשנתבקש על סרט על מלחמת העולם השנייה. לאחר שהמשתמש תיקן אותו וביקש "רק מלחמת העולם השנייה", הצ\'אטבוט המליץ על סרט שעוסק במלחמת העולם הראשונה, תוך שהוא מציין זאת בעצמו. זוהי טעות חמורה בהבנת ההקשר והוראות המשתמש.', politeness=5, knowledge_from_RAG=True, answer_relevance=1, not_reveal_system_info=True, hallucination=False)

In [24]:
temp.parsed.feedback

'הצ\'אטבוט נכשל בהבנת הבקשה של המשתמש. בתשובה הראשונה, הוא המליץ על סרט על מלחמת יום כיפור כשנתבקש על סרט על מלחמת העולם השנייה. לאחר שהמשתמש תיקן אותו וביקש "רק מלחמת העולם השנייה", הצ\'אטבוט המליץ על סרט שעוסק במלחמת העולם הראשונה, תוך שהוא מציין זאת בעצמו. זוהי טעות חמורה בהבנת ההקשר והוראות המשתמש.'

# Dashboard

In [ ]:
# Calculate text lengths
df['question_length'] = df[df['campaign_name'] == 'open question']['campaign_details'].str.split().str.len()
df['answer_length'] = df[df['campaign_name'] == 'response answer']['campaign_details'].str.split().str.len()

# Create separate dataframes for questions and answers
questions_df = df[df['campaign_name'] == 'open question'].copy()
answers_df = df[df['campaign_name'] == 'response answer'].copy()

# Create subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('התפלגות אורך שאלות משתמשים', 'התפלגות אורך תשובות המודל',
                   'Histogram - אורך שאלות', 'Histogram - אורך תשובות'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Box plot for questions
fig.add_trace(
    go.Box(y=questions_df['question_length'].dropna(), 
           name='אורך שאלות',
           boxpoints='outliers'),
    row=1, col=1
)

# Box plot for answers  
fig.add_trace(
    go.Box(y=answers_df['answer_length'].dropna(),
           name='אורך תשובות',
           boxpoints='outliers'),
    row=1, col=2
)

# Histogram for questions
fig.add_trace(
    go.Histogram(x=questions_df['question_length'].dropna(),
                name='שאלות',
                nbinsx=30,
                opacity=0.7),
    row=2, col=1
)

# Histogram for answers
fig.add_trace(
    go.Histogram(x=answers_df['answer_length'].dropna(),
                name='תשובות', 
                nbinsx=30,
                opacity=0.7),
    row=2, col=2
)

# Update layout
fig.update_layout(
    height=800,
    title_text="התפלגות אורך טקסט - שאלות ותשובות",
    title_x=0.5,
    showlegend=False
)

# Update axes labels
fig.update_xaxes(title_text="אורך תווים", row=2, col=1)
fig.update_xaxes(title_text="אורך תווים", row=2, col=2)
fig.update_yaxes(title_text="אורך תווים", row=1, col=1)
fig.update_yaxes(title_text="אורך תווים", row=1, col=2)
fig.update_yaxes(title_text="תדירות", row=2, col=1)
fig.update_yaxes(title_text="תדירות", row=2, col=2)

fig.show()

# # Print summary statistics
# print("סטטיסטיקות אורך שאלות:")
# print(questions_df['question_length'].describe())
# print("\nסטטיסטיקות אורך תשובות:")
# print(answers_df['answer_length'].describe())




סטטיסטיקות אורך שאלות:
count    13.000000
mean      3.230769
std       2.127355
min       1.000000
25%       2.000000
50%       3.000000
75%       3.000000
max       8.000000
Name: question_length, dtype: float64

סטטיסטיקות אורך תשובות:
count     8.000000
mean     70.750000
std      20.878903
min      46.000000
25%      52.750000
50%      70.000000
75%      87.500000
max      95.000000
Name: answer_length, dtype: float64


In [43]:
df

,sso_id,campaign_details,campaign_name,platform,event_time,record_type,index,session_id,total_time,rag_speed,...,remaining_user_messages,troll_triggered,conversation_key,genres,regenerate,media_type,streaming_platforms,id,question_length,answer_length
0,8010962861,היוש,open question,Desktop,2025-06-30 08:31:35,action,1,151,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8010962861_151,1.0,NaN
1,9037014398,אפשר דוקו טוב?,open question,App,2025-06-30 08:31:01,action,4,067b528f-b885-4d36-be53-a9dccb6f6483,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9037014398_067b528f-b885-4d36-be53-a9dccb6f6483,3.0,NaN
2,50000227213,2 random films,open question,Desktop,2025-06-30 08:12:39,action,6,16,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50000227213_16,3.0,NaN
3,50000227213,2 random films,open question,Desktop,2025-06-30 08:12:28,action,9,16,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50000227213_16,3.0,NaN
4,50000227213,2 random films,open question,Desktop,2025-06-30 08:08:51,action,10,15,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50000227213_15,3.0,NaN
5,50000227213,2 random films,open question,Desktop,2025-06-30 08:08:36,action,11,15,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50000227213_15,3.0,NaN
6,9037857899,מלחמת העולם השניה,open question,Mobile,2025-06-30 08:07:29,action,14,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9037857899_4,3.0,NaN
7,8010962861,הי,open question,Desktop,2025-06-30 07:20:15,action,18,151,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8010962861_151,1.0,NaN
8,8010962861,הי,open question,Desktop,2025-06-30 07:19:17,action,20,150,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8010962861_150,1.0,NaN
9,9037857899,רק מלחמת העולם השניה לא מלחמות אחרות,open question,Desktop,2025-06-30 06:01:31,action,23,7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9037857899_7,7.0,NaN


# גרפים נוספים שאפשר להכין

בהתבסס על הנתונים שיש לנו, הנה רשימה של גרפים נוספים שאפשר לבצע:

## 1. ניתוח זמני
- **התפלגות לפי שעות ביום** - מתי משתמשים הכי פעילים
- **התפלגות לפי ימים בשבוע** - איזה ימים יש הכי הרבה פעילות
- **זמן תגובה של המודל** - כמה זמן לוקח למודל להגיב

## 2. ניתוח משתמשים
- **חלוקה לפי פלטפורמה** (platform)
- **התפלגות פעילות לפי SSO ID** - אילו משתמשים הכי פעילים
- **אורך שיחות לפי משתמש** - מי מנהל שיחות ארוכות יותר

## 3. ניתוח תוכן
- **מילות מפתח נפוצות** - ענן מילים של השאלות הנפוצות
- **התפלגות נושאים** - ניתוח נושאי השאלות (סרטים, סדרות, ז'אנרים)
- **סנטימנט של השאלות** - האם השאלות חיוביות/שליליות/ניטרליות

## 4. ניתוח איכות עם LLM Judge
- **התפלגות ציוני איכות** (quality scores)
- **התפלגות ציוני אדיבות** (politeness scores)
- **התפלגות רלוונטיות התשובות** (answer_relevance)
- **שיעור השימוש ב-RAG** (knowledge_from_RAG)
- **שיעור הזיות** (hallucination rate)

## 5. ניתוח ביצועים
- **יחס אורך תשובה לאורך שאלה** - האם תשובות ארוכות יותר לשאלות מורכבות
- **התפלגות מספר החליפות בשיחה** - כמה הודעות בממוצע בכל שיחה
- **זמן בין שאלות באותה שיחה** - כמה זמן משתמשים מחכים לפני שאלה נוספת

In [ ]:
# דוגמאות לגרפים שאפשר ליישם מיד עם הנתונים הקיימים:

# 1. גרף התפלגות לפי פלטפורמה
platform_counts = df['platform'].value_counts()
fig_platform = px.pie(
    values=platform_counts.values, 
    names=platform_counts.index,
    title="התפלגות לפי פלטפורמה"
)
fig_platform.show()

# 2. התפלגות פעילות לפי שעה ביום
df['hour'] = pd.to_datetime(df['event_time']).dt.hour
hourly_activity = df['hour'].value_counts().sort_index()

fig_hourly = px.bar(
    x=hourly_activity.index,
    y=hourly_activity.values,
    title="פעילות לפי שעה ביום",
    labels={'x': 'שעה', 'y': 'מספר הודעות'}
)
fig_hourly.show()

# 3. התפלגות משתמשים לפי כמות הודעות
user_activity = df['sso_id'].value_counts()
fig_user_activity = px.histogram(
    x=user_activity.values,
    nbins=20,
    title="התפלגות משתמשים לפי כמות הודעות",
    labels={'x': 'מספר הודעות', 'y': 'מספר משתמשים'}
)
fig_user_activity.show()

# 4. יחס אורך תשובה לאורך שאלה
questions_with_answers = df[df['campaign_name'] == 'open question']['question_length'].dropna()
answers_with_questions = df[df['campaign_name'] == 'response answer']['answer_length'].dropna()

if len(questions_with_answers) > 0 and len(answers_with_questions) > 0:
    fig_comparison = go.Figure()
    fig_comparison.add_trace(go.Scatter(
        x=questions_with_answers.values,
        y=answers_with_questions.values,
        mode='markers',
        name='אורך תשובה vs שאלה',
        opacity=0.6
    ))
    fig_comparison.update_layout(
        title="יחס אורך תשובה לאורך שאלה",
        xaxis_title="אורך שאלה (מילים)",
        yaxis_title="אורך תשובה (מילים)"
    )
    fig_comparison.show()

# 5. מספר הודעות לפי משתמש (Top 10)
top_users = df['sso_id'].value_counts().head(10)
fig_top_users = px.bar(
    x=top_users.index.astype(str),
    y=top_users.values,
    title="10 המשתמשים הכי פעילים",
    labels={'x': 'SSO ID', 'y': 'מספר הודעות'}
)
fig_top_users.show()

In [ ]:
# גרפים מתקדמים נוספים:

# 6. מטריקס זמני - פעילות לפי יום ושעה (Heatmap)
df['date'] = pd.to_datetime(df['event_time']).dt.date
df['day_of_week'] = pd.to_datetime(df['event_time']).dt.day_name()
daily_hourly = df.groupby(['day_of_week', 'hour']).size().reset_index(name='count')

# יצירת heatmap
pivot_table = daily_hourly.pivot(index='day_of_week', columns='hour', values='count').fillna(0)
fig_heatmap = px.imshow(
    pivot_table,
    title="מטריקס פעילות - יום ושעה",
    labels=dict(x="שעה", y="יום בשבוע", color="מספר הודעות"),
    color_continuous_scale="Blues"
)
fig_heatmap.show()

# 7. התפלגות אורך שיחות (מספר הודעות לכל שיחה)
conversation_lengths = df.groupby('id').size()
fig_conv_length = px.histogram(
    x=conversation_lengths.values,
    nbins=20,
    title="התפלגות אורך שיחות (מספר הודעות)",
    labels={'x': 'מספר הודעות בשיחה', 'y': 'מספר שיחות'}
)
fig_conv_length.show()

# 8. יחס שאלות לתשובות
question_count = len(df[df['campaign_name'] == 'open question'])
answer_count = len(df[df['campaign_name'] == 'response answer'])

fig_ratio = px.pie(
    values=[question_count, answer_count],
    names=['שאלות משתמשים', 'תשובות מודל'],
    title="יחס שאלות לתשובות"
)
fig_ratio.show()

# 9. התפלגות זמנית - פעילות לפי תאריך
daily_activity = df.groupby('date').size().reset_index(name='activity')
daily_activity['date'] = pd.to_datetime(daily_activity['date'])

fig_daily = px.line(
    daily_activity,
    x='date',
    y='activity',
    title="פעילות יומית",
    labels={'date': 'תאריך', 'activity': 'מספר הודעות'}
)
fig_daily.show()

# 10. סטטיסטיקות תיאוריות
print("סטטיסטיקות כלליות:")
print(f"מספר כולל של הודעות: {len(df)}")
print(f"מספר שיחות ייחודיות: {df['id'].nunique()}")
print(f"מספר משתמשים ייחודיים: {df['sso_id'].nunique()}")
print(f"אורך שאלה ממוצע: {df[df['campaign_name'] == 'open question']['question_length'].mean():.1f} מילים")
print(f"אורך תשובה ממוצע: {df[df['campaign_name'] == 'response answer']['answer_length'].mean():.1f} מילים")

# גרפים נוספים שדורשים ניתוח תוכן מתקדם

## 11. ניתוח מילות מפתח ותוכן
```python
# התפלגות מילות מפתח נפוצות בשאלות
from collections import Counter
import re

# הפרדת מילים מהשאלות
questions_text = df[df['campaign_name'] == 'open question']['campaign_details'].fillna('')
all_words = []
for text in questions_text:
    words = re.findall(r'\b\w+\b', text.lower())
    all_words.extend(words)

# מילות המפתח הנפוצות ביותר
word_counts = Counter(all_words)
top_words = word_counts.most_common(20)

# גרף מילות מפתח
fig_words = px.bar(
    x=[word for word, count in top_words],
    y=[count for word, count in top_words],
    title="20 המילים הנפוצות ביותר בשאלות"
)
fig_words.show()
```

## 12. ניתוח נושאים
```python
# זיהוי נושאים שכיחים (סרטים, סדרות, ז'אנרים)
movie_keywords = ['סרט', 'סרטים', 'movie', 'film']
series_keywords = ['סדרה', 'סדרות', 'series', 'show']
genre_keywords = ['קומדיה', 'דרמה', 'אקשן', 'רומנטי', 'מותחן']

def categorize_topic(text):
    text_lower = text.lower()
    if any(word in text_lower for word in movie_keywords):
        return 'סרטים'
    elif any(word in text_lower for word in series_keywords):
        return 'סדרות'
    elif any(word in text_lower for word in genre_keywords):
        return 'ז'אנרים'
    else:
        return 'אחר'

df['topic_category'] = df[df['campaign_name'] == 'open question']['campaign_details'].apply(categorize_topic)
topic_distribution = df['topic_category'].value_counts()

fig_topics = px.pie(values=topic_distribution.values, names=topic_distribution.index,
                   title="התפלגות נושאי השאלות")
fig_topics.show()
```

## 13. ניתוח איכות עם LLM Judge (אם נריץ על כל השיחות)
```python
# אם נריץ את ה-LLM Judge על כל השיחות, נוכל לבצע:
# - התפלגות ציוני איכות
# - התפלגות ציוני אדיבות
# - שיעור השימוש ב-RAG
# - שיעור הזיות
# - קורלציה בין אורך תשובה לאיכות
```

## 14. ניתוח זמני מתקדם
```python
# זמן תגובה בין שאלה לתשובה
# מגמות לאורך זמן
# פעילות בשעות שיא
# השוואה בין ימי חול לסוף שבוע
```